In [1]:
import numpy as np
from keras.datasets import mnist
from keras.layers import  (Activation, BatchNormalization, Dense, Dropout, Flatten, Reshape)
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.models import Sequential
from keras.optimizers import Adam

img_rows=28
img_cols=28
channels=1

img_shape=(img_rows, img_cols, channels)

z_dim=100

**Build Generator**

In [12]:
def build_generator(z_dim):
  model=Sequential()
  model.add(Dense(256*7*7,input_dim=z_dim))
  model.add(Reshape((7,7,256) , input_dim=z_dim))
  model.add(Conv2DTranspose(128,kernel_size=3,strides=2,padding='same'))
  model.add(BatchNormalization())
  model.add(LeakyReLU(alpha=0.01))
  model.add(Conv2DTranspose(1,kernel_size=3,strides=2,padding='same'))
  model.add(Activation('tanh'))

  return model

**Build Discriminator**

In [8]:
def build_discriminator(img_shape):
  model=Sequential()
  model.add(Conv2D(32,kernel_size=3,strides=2,input_shape=img_shape,padding='same'))
  model.add(LeakyReLU(alpha=0.01))
  model.add(Conv2D(64,kernel_size=3,strides=2,input_shape=img_shape,padding='same'))
  model.add(BatchNormalization())
  model.add(LeakyReLU(alpha=0.01))
  model.add(Conv2D(128,kernel_size=3,strides=2,input_shape=img_shape,padding='same'))
  model.add(BatchNormalization())
  model.add(LeakyReLU(alpha=0.01))
  model.add(Flatten())
  model.add(Dense(1,activation='sigmoid'))

  return model

**Build GAN**

In [13]:
def build_gan(generator,discriminator):
  model=Sequential()
  model.add(generator)
  model.add(discriminator)
  return model

discriminator=build_discriminator(img_shape)
discriminator.compile(loss='binary_crossentropy',optimizer=Adam(),metrics=['accuracy'])

generator=build_generator(z_dim)
discriminator.trainable=False

gan=build_gan(generator,discriminator)
gan.compile(loss='binary_crossentropy',optimizer=Adam())


**GAN Training**

In [19]:
losses=[]
accuracies=[]
iteration_checkpoints=[]

def train(iterations,batch_size,sample_interval):
  (X_train,_),(_,_) = mnist.load_data()
  X_train = X_train / 127.50 - 1.0
  X_train=np.expand_dims(X_train,axis=3)

  real = np.ones((batch_size,1))
  fake = np.zeros((batch_size,1))

  for iteration in range(iterations):
    idx=np.random.randint(0,X_train.shape[0],batch_size)
    imgs=X_train[idx]

    z=np.random.normal(0,1,(batch_size,100))
    gen_imgs=generator.predict(z)

    d_loss_real=discriminator.train_on_batch(imgs,real)
    d_loss_fake=discriminator.train_on_batch(gen_imgs,fake)
    d_loss,accuracy=0.5*np.add(d_loss_real,d_loss_fake)

    z=np.random.normal(0,1,(batch_size,100))
    gen_imgs=generator.predict(z)

    g_loss=gan.train_on_batch(z,real)

    if (iteration+1) % sample_interval == 0:
      losses.append((d_loss, g_loss))                            
      accuracies.append(100.0 * accuracy)
      iteration_checkpoints.append(iteration + 1)
      print("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (iteration + 1, d_loss, 100.0 * accuracy, g_loss))

**Run Model**

In [22]:
iterations=2000
batch_size=128
sample_interval=50

train(iterations, batch_size, sample_interval)

50 [D loss: 0.000023, acc.: 100.00%] [G loss: 0.030996]
100 [D loss: 0.000018, acc.: 100.00%] [G loss: 0.035417]
150 [D loss: 0.000023, acc.: 100.00%] [G loss: 0.036105]
200 [D loss: 0.000020, acc.: 100.00%] [G loss: 0.029588]
250 [D loss: 0.000029, acc.: 100.00%] [G loss: 0.026443]
300 [D loss: 0.000087, acc.: 100.00%] [G loss: 0.027757]
350 [D loss: 0.000012, acc.: 100.00%] [G loss: 0.022515]
400 [D loss: 0.000023, acc.: 100.00%] [G loss: 0.020867]
450 [D loss: 0.000013, acc.: 100.00%] [G loss: 0.024715]
500 [D loss: 0.000015, acc.: 100.00%] [G loss: 0.016334]
550 [D loss: 0.000008, acc.: 100.00%] [G loss: 0.017785]
600 [D loss: 0.000009, acc.: 100.00%] [G loss: 0.018248]
650 [D loss: 0.000004, acc.: 100.00%] [G loss: 0.019698]
700 [D loss: 0.000005, acc.: 100.00%] [G loss: 0.024148]
750 [D loss: 0.000003, acc.: 100.00%] [G loss: 0.020614]
800 [D loss: 0.000006, acc.: 100.00%] [G loss: 0.013825]
850 [D loss: 0.000006, acc.: 100.00%] [G loss: 0.013743]
900 [D loss: 0.000004, acc.: 100